# Collaboration and Competition - Tennis

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

import torch
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque
import time


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

When finished, you can close the environment.

### 2. Train the agent

When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
my_configs = [  {
        "n_episodes": 2500,
        "seeds": 1,
        "learn_every": 5,
        "hidden_units": 128,
        "lr_critic": 0.001,
        "lr_actor": 0.0001,
        "weight_deacy": 1,
        "learn_num": 5,
        "gamma": 0.99,
        "tau": 0.07,
        "ou_sigma": 0.3,
        "ou_theta": 0.15,
        "eps_start": 1.0,
        "eps_ep_end": 400,
        "eps_final": 0.1,
        "batch_size": 256
    }]

import train
train.train(my_configs, "tmp", ids=[0])

Nmber of configs: 1


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: 
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]

Prepare for new configuration. Make new MADDPG agent.
Now running with config: {'n_episodes': 2500, 'seeds': 1, 'learn_every': 5, 'hidden_units': 128, 'lr_critic': 0.001, 'lr_actor': 0.0001, 'weight_deacy': 1, 'learn_num': 5, 'gamma': 0.99, 'tau': 0.07, 'ou_sigma': 0.3, 'ou_theta': 0.15, 'eps_start': 1.0, 'eps_ep_end': 400, 'eps_final': 0.1, 'batch_size': 256}
Prepare for new configuration. Make new MADDPG agent.
Episode 10	Average Score: 0.00	Current Score: 0.0
Noise Scaling: 1.0, Memory size: 169 and Num Steps: 31
Episode 20	Average Score: 0.01	Current Score: 0.0
Noise Scaling

Episode 610	Average Score: 0.04	Current Score: 0.10000000149011612
Noise Scaling: 0.6003423698268051, Memory size: 12115 and Num Steps: 31
Episode 620	Average Score: 0.04	Current Score: 0.0
Noise Scaling: 0.5943658896200157, Memory size: 12377 and Num Steps: 14
Episode 630	Average Score: 0.04	Current Score: 0.0
Noise Scaling: 0.5884489059896089, Memory size: 12570 and Num Steps: 14
Episode 640	Average Score: 0.05	Current Score: 0.10000000149011612
Noise Scaling: 0.5825908266400397, Memory size: 12884 and Num Steps: 30
Episode 650	Average Score: 0.05	Current Score: 0.10000000149011612
Noise Scaling: 0.5767910651721362, Memory size: 13125 and Num Steps: 30
Episode 660	Average Score: 0.05	Current Score: 0.0
Noise Scaling: 0.5710490410244007, Memory size: 13393 and Num Steps: 14
Episode 670	Average Score: 0.06	Current Score: 0.0
Noise Scaling: 0.5653641794148944, Memory size: 13663 and Num Steps: 14
Episode 680	Average Score: 0.06	Current Score: 0.10000000149011612
Noise Scaling: 0.5597359

In [ ]:
from IPython.display import display
from PIL import Image
path="results/plot_3.png"
display(Image.open(path))
